In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mesa import Agent, Model
from mesa.space import SingleGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import random
import functools

# Mesa Example

In [ ]:
def calculate_total_population(model):
    return model.num_agents

def population_fraction(clone, model):
    N = model.num_agents
    agent_list = self.model.schedule.agents
    N_clone = 0
    for agent in agent_list:
        if agent.clone == clone:
            N_clone += 1
    return N_clone/N

In [ ]:
class SCGrower(Agent):
    def __init__(self, unique_id, model, clone):
        super().__init__(unique_id, model)
        self.clone = clone
    
    def step(self):
        neighbourhood = [pos for pos in self.model.grid.iter_neighborhood(self.pos, True)]
        random.shuffle(neighbourhood)
        for pos in neighbourhood:
            if self.model.grid.is_cell_empty(pos):
                A = SCGrower(self.model.num_agents, self.model, self.clone)
                self.model.num_agents += 1
                self.model.grid.place_agent(A, pos)
                self.model.schedule.add(A)
                break
                
class SCGrowthModel(Model):
    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = SingleGrid(height, width, False)
        self.schedule = RandomActivation(self)
        model_reporters = {"Population": calculate_total_population}
        for i in range(self.num_agents):
            a = SCGrower(i, self, i)
            self.schedule.add(a)
            self.grid.position_agent(a)
        
        self.datacollector = DataCollector(model_reporters=model_reporters)
        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)

In [ ]:
scg = SCGrowthModel(5, 10, 10)
for i in range(10):
    scg.step()
pop = scg.datacollector.get_model_vars_dataframe()
print(pop)

# Interactive visualisation

In [ ]:
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter

def agent_portrayal(agent):
    portrayal = {"Shape": "circle", "Filled": "true", "r": 0.5, "Layer": 0}
    colours = ["red", "blue", "green", "yellow", "purple", "orange", "brown", "grey", "black", "cyan"]
    portrayal["Color"] = colours[agent.clone]
    return portrayal

grid = CanvasGrid(agent_portrayal, 40, 40, 500, 500)
chart = ChartModule([
    {"Label": "Population", "Color": "#0000FF"}],
    data_collector_name='datacollector'
)

model_params = {"N": UserSettableParameter('slider', "Number of agents", 5, 1, 10, 1,
                               description="Choose how many agents to include in the model"), "width": 40, "height": 40}
server = ModularServer(SCGrowthModel, [grid, chart], "Spatially-Constrained Growth", model_params)
server.port = 8521
server.launch()

# Batch Running

In [ ]:
from mesa.batchrunner import BatchRunner

fixed_params = {"N": 5}
variable_params = {"width": range(5, 20, 5), "height": range(5, 20, 5)}

batch_run = BatchRunner(SCGrowthModel,
                        fixed_parameters=fixed_params,
                        variable_parameters=variable_params,
                        iterations=5,
                        max_steps=100,
                        model_reporters={"Population": calculate_total_population})
batch_run.run_all()


# Ecological model

In [ ]:
class Predator(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.label = "predator"
        self.probdiv = 0.15  # probability of division
        self.probdie = 0.2   # probability of death each turn
        
    def step(self):
        neighbours = self.model.grid.get_neighbors(self.pos, 1, False)
        neighbourhood = [pos for pos in self.model.grid.iter_neighborhood(self.pos, True)]
        random.shuffle(neighbourhood)  # randomly resort the neighboring positions on grid
        
        if_die = np.random.choice([1, 0], size = (1), p=[self.probdie, 1 - self.probdie])
        if (bool(if_die)):
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
        else:
            if_div = np.random.choice([1, 0], size = (1), p=[self.probdiv, 1 - self.probdiv])
            if (bool(if_div)):
                self.probdiv = 0.25
                for pos in neighbourhood:
                    if self.model.grid.is_cell_empty(pos):
                        A = Predator(self.model.num_agents, self.model)
                        self.model.num_agents += 1
                        self.model.grid.place_agent(A, pos)
                        self.model.schedule.add(A)
                        break

            for neighbor in neighbours:
                if neighbor.label == "prey":
                    self.model.grid.remove_agent(neighbor)
                    self.model.schedule.remove(neighbor)

                    self.probdiv = min(2*self.probdiv, 1)
        
class Prey(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.label = "prey"

    def step(self):
        neighbours = self.model.grid.get_neighbors(self.pos, 1, False)
        neighbourhood = [pos for pos in self.model.grid.iter_neighborhood(self.pos, True)]
        random.shuffle(neighbourhood)  # randomly resort the neighboring positions on grid
        for pos in neighbourhood:
            if self.model.grid.is_cell_empty(pos):
                A = Prey(self.model.num_agents, self.model)
                self.model.num_agents += 1
                self.model.grid.place_agent(A, pos)
                self.model.schedule.add(A)
                break
    
class Mutual1(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.label = "mutual1"
        """
        Initialize state properties
        """
    def step(self):
        neighbours = self.model.grid.get_neighbors(self.pos, 1, False)
        """
        Do something based on neighbours, e.g. 
        self.model.grid.position_agent(agent, x, y)
        self.model.grid.remove_agent(agent)
        """
        pass

class Mutual2(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.label = "mutual2"
        """
        Initialize state properties
        """
    def step(self):
        neighbours = self.model.grid.get_neighbors(self.pos, 1, False)
        """
        Do something based on neighbours, e.g. 
        self.model.grid.position_agent(agent, x, y)
        self.model.grid.remove_agent(agent)
        """
        pass

In [ ]:
class PredatorPreyModel(Model):
    def __init__(self, N_pred, N_prey, width, height):
        self.num_pred = N_pred
        self.num_prey = N_prey
        self.num_agents = N_pred + N_prey
        self.grid = SingleGrid(height, width, False)
        self.schedule = RandomActivation(self)
        model_reporters = {"Population": calculate_total_population}
        for i in range(self.num_pred):
            a = Predator(i, self)
            self.schedule.add(a)
            self.grid.position_agent(a)
        for i in range(self.num_prey):
            a = Prey(i+N_pred, self)
            self.schedule.add(a)
            self.grid.position_agent(a)
            
        self.datacollector = DataCollector(model_reporters=model_reporters)
        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        self.datacollector.collect(self)
        
scg = PredatorPreyModel(5, 5, 10, 10)
for i in range(10):
    scg.step()
pop = scg.datacollector.get_model_vars_dataframe()
print(pop)


In [ ]:
from mesa.visualization.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.UserParam import UserSettableParameter

def agent_portrayal(agent):
    portrayal = {"Shape": "circle", "Filled": "true", "r": 0.5, "Layer": 0}
    colours = ["red", "blue", "green", "yellow", "purple", "orange", "brown", "grey", "black", "cyan"]
    portrayal["Color"] = colours[agent.label=="prey"]
    return portrayal

grid = CanvasGrid(agent_portrayal, 40, 40, 500, 500)
chart = ChartModule([
    {"Label": "Population", "Color": "#0000FF"}],
    data_collector_name='datacollector'
)

model_params = {"N_pred": UserSettableParameter('slider', "Number of predators", 5, 1, 10, 1),
                "N_prey": UserSettableParameter('slider', "Number of prey", 5, 1, 10, 1), "width": 40, "height": 40}
server = ModularServer(PredatorPreyModel, [grid, chart], "Spatially-Constrained Growth", model_params)
server.port = 8521
server.launch()